In [1]:
from clustertools import build_datacube
from clustertools.parameterset import build_parameter_set

from pprint import pprint
import numpy as np
from train_monuseg_hard_clustertools import weight_exclude, exclude_target_and_dice_calibration
from train_monuseg_hard_clustertools import no_distillation_filter, filter_nc_rr, min_weight_only_for_entropy, restrict_fn_absolute
from train_monuseg_baseline_clustertools import min_weight_for_entropy, not_using_incomplete
from ipnb_helpers import ExperimentReader, plot_table

In [2]:
import __main__
setattr(__main__, "weight_exclude", weight_exclude)
setattr(__main__, "exclude_target_and_dice_calibration", exclude_target_and_dice_calibration)
setattr(__main__, "no_distillation_filter", no_distillation_filter)
setattr(__main__, "filter_nc_rr", filter_nc_rr)
setattr(__main__, "min_weight_only_for_entropy", min_weight_only_for_entropy)
setattr(__main__, "min_weight_for_entropy", min_weight_for_entropy)
setattr(__main__, "not_using_incomplete", not_using_incomplete)
setattr(__main__, "restrict_fn_absolute", restrict_fn_absolute)

In [ ]:
baseline = ExperimentReader("monuseg-unet-baseline", "monuseg-reeval-baseline", "rseed")
hard_exp = ExperimentReader("monuseg-unet-hard", "monuseg-reeval-hard", "monu_ms")
nogt_exp = ExperimentReader("monuseg-unet-nogt", "monuseg-reeval-nogt", "monu_ms")


In [ ]:
rows = [
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "constant", "weights_constant": '0.05'},
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "constant", "weights_constant": '0.2'},
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "constant", "weights_constant": '0.5'},
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "constant", "weights_constant": '1.0'},
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "balance_gt" },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_entropy", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_entropy", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_consistency", "weights_neighbourhood": '1', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_consistency", "weights_neighbourhood": '1', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_consistency", "weights_neighbourhood": '2', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_consistency", "weights_neighbourhood": '2', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "absolute", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "quadratic", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "absolute", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "quadratic", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "constant", "weights_constant": '0.05'},
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "constant", "weights_constant": '0.2'},
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "constant", "weights_constant": '0.5'},
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "constant", "weights_constant": '1.0'},
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "balance_gt" },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_entropy", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_entropy", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_consistency", "weights_neighbourhood": '1', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_consistency", "weights_neighbourhood": '1', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_consistency", "weights_neighbourhood": '2', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_consistency", "weights_neighbourhood": '2', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "absolute", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "quadratic", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "absolute", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "quadratic", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'True' },
]


In [ ]:
hard_exp_fn = lambda **kwargs: hard_exp.get_reeval_metric("test_hard_dice", **kwargs)
nogt_exp_fn = lambda **kwargs: nogt_exp.get_metric("val_dice", **kwargs)
columns = [
    (nogt_exp_fn, {"monu_nc": '1', "monu_rr": '1.0', "n_calibration": '0'}),
    (nogt_exp_fn, {"monu_nc": '2', "monu_rr": '1.0', "n_calibration": '1'}),
    (hard_exp_fn, {"monu_nc": '1', "monu_rr": '0.9', "n_calibration": '0'}),
    (hard_exp_fn, {"monu_nc": '2', "monu_rr": '0.9', "n_calibration": '1'}),
    (hard_exp_fn, {"monu_nc": '2', "monu_rr": '0.5', "n_calibration": '0'}),
    (hard_exp_fn, {"monu_nc": '3', "monu_rr": '0.5', "n_calibration": '1'}),
    (hard_exp_fn, {"monu_nc": '4', "monu_rr": '0.25', "n_calibration": '0'}),
    (hard_exp_fn, {"monu_nc": '5', "monu_rr": '0.25', "n_calibration": '1'})
]


In [ ]:
plot_table(rows, columns, 30)


In [ ]:
hard_exp._cube.domain

In [ ]:
hard_exp._cube(
    monu_nc="4", 
    n_calibration="0", 
    monu_rr="0.5", 
    no_distillation="False", 
    weights_mode="pred_merged", 
    weights_neighbourhood="2",
    distil_target_mode='soft',
    weights_minimum="0.0").diagnose()

In [ ]:
bl = baseline.get_reeval_metric("test_hard_dice", **{})
bl_avg, bl_std = np.mean(bl, axis=0), np.std(bl, axis=0)
bl_best = np.argmax(bl_avg)
print("{:2.2f} ± {:2.2f}".format(bl_avg[bl_best] * 100, bl_std[bl_best] * 100))